In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fhir_client import FHIRClient
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
client = FHIRClient(service_base_url='http://localhost:8080/baseDstu3')
pats = client.get_all_patients()

In [4]:
from ml_on_fhir import MLOnFHIR
from fhir_objects.Patient import Patient
from sklearn.tree import DecisionTreeClassifier

ml_fhir = MLOnFHIR(Patient, feature_attrs=['birthDate'], label_attrs=['gender'])
X, y, trained_clf = ml_fhir.fit(pats, DecisionTreeClassifier())

from sklearn.metrics import accuracy_score, roc_curve, auc
fpr, tpr, _ = roc_curve(y, trained_clf.predict(X))
print("Prediction accuracy {}".format( auc(fpr, tpr) ) )

INFO:root:Extracting attributes from data set
INFO:root:Preprocessing data
INFO:root:Started training of clf
INFO:root:Training completed


Prediction accuracy 0.8989813242784381


#### The first five values of the ` birthDate` feature that has been preprocessed into an age in years:

In [8]:
X[:5]

array([[67],
       [15],
       [84],
       [85],
       [84]])

### Custom Preprocessing Classes
If you want to preprocess fhir resources differently, feel free to implement your own preprocessing class. 
We can use the `register_preprocessor` function to do so. It is crucial to stick to the following naming scheme: 
    
    "<FHIR_Object_Name><FHIR_Resource_Name>Processor"
    
Furthemore, at least the methods `fit` and `transform` have to be implemented.

In [5]:
from preprocessing import register_preprocessor

from sklearn.base import BaseEstimator
import datetime as dt
import numpy as np
from fhir_resources import date_format

class PatientBirthdateProcessor(BaseEstimator):
    """
    Calculates the age in days to use birthdate as a feature 
    """
    def transform(self, X, **transform_params):
        ages = []
        for birthdate in X:
            b_date = dt.datetime.strptime(birthdate[0], date_format)
            ages.append([int(
                            (dt.datetime.now().date() - b_date.date()).days)])
        return np.array(ages)

    def fit(self, X, y=None, **fit_params):
        return self
    
register_preprocessor(PatientBirthdateProcessor)

In [6]:
ml_fhir = MLOnFHIR(Patient, feature_attrs=['birthDate'], label_attrs=['gender'])
X, y, trained_clf = ml_fhir.fit(pats, DecisionTreeClassifier())

INFO:root:Extracting attributes from data set
INFO:root:Preprocessing data
INFO:root:Started training of clf
INFO:root:Training completed


#### The `birthDate` feature is now the age in days:

In [13]:
X[:5]

array([[24705],
       [ 5802],
       [30961],
       [31046],
       [30950]])